In [1]:
# 구현하려는 것
# https://github.com/vinhkhuc/MemN2N-babi-python
# https://github.com/nmhkahn/MemN2N-pytorch

In [2]:
from __future__ import print_function
from utils import *

In [3]:
import mxnet as mx
ctx = mx.cpu()

In [4]:
import os
import glob
data_dir = glob.glob(os.getcwd() + '/data/')[0]

!cd /home/hyungjunkim/Dropbox/repo/dl/MemN2N/scripts/data && \  
wget https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz && \  
tar xvf babi_tasks_1-20_v1-2.tar.gz

In [5]:
from datautils import *

In [7]:
train_data = bAbIDataset(data_dir + 'tasks_1-20_v1-2/en/', task_id=1, train=True)

/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [8]:
test_data = bAbIDataset(data_dir + 'tasks_1-20_v1-2/en/', task_id=1, train=False)

/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [9]:
word_re = {value:key for key, value in train_data.word_idx.items()}

In [10]:
word_re 

{1: 'back',
 2: 'bathroom',
 3: 'bedroom',
 4: 'daniel',
 5: 'garden',
 6: 'hallway',
 7: 'is',
 8: 'john',
 9: 'journeyed',
 10: 'kitchen',
 11: 'mary',
 12: 'moved',
 13: 'office',
 14: 'sandra',
 15: 'the',
 16: 'to',
 17: 'travelled',
 18: 'went',
 19: 'where'}

In [11]:
for x,y,z in train_data:
    for idx in x:
        print (idx)
        print (" ".join([word_re.get(int(i), '.') for i in idx if int(i) != 0]))
    
    print (" ".join([word_re.get(int(i), '?') for i in y if int(i) != 0]))
    print (word_re.get(int(z), '?'))
    break

[11 12 16 15  2 11]
mary moved to the bathroom mary
[ 8 18 16 15  6 10]
john went to the hallway kitchen
[0 0 0 0 0 0]

[0 0 0 0 0 0]

[0 0 0 0 0 0]

[0 0 0 0 0 0]

[0 0 0 0 0 0]

[0 0 0 0 0 0]

[0 0 0 0 0 0]

[0 0 0 0 0 0]

where is mary
bathroom


In [12]:
inputs_train, queries_train, answers_train = train_data.data_story, train_data.data_query, train_data.data_answer
inputs_test, queries_test, answers_test = test_data.data_story, test_data.data_query, test_data.data_answer

In [13]:
def position_encoding(sentences, emb_dim):
    s_tmp = sentences.asnumpy()
    n_row,n_col = np.shape(s_tmp)
    for i in range(n_row):
        for j in range(n_col):
            s_tmp[i,j] = (1-j/n_col) - (n_row/emb_dim)*(1-2*j/n_col)    
    return mx.nd.array(s_tmp, ctx).expand_dims(2)

## End-To-End Memory Network (N2NMN)

i = 문장   
j = 단어

Inputs: $x_{i}$: 문장(context)  
$q$: 질의  
$target$: 답변(a word or words)

![img0](imgs/n2nmn_fig0.png)

#### 메모리 벡터: 임베딩된 워드의 합을 문장으로 표기

$m_{i}={\sum}_{j}Ax_{i,j}$

#### Attention $P_{i}$: 질의(q)와 가장 관련된 Senence(context) 확률로 나타냄

$P_i=Softmax(uT {\cdot} mi)$

$o={\sum}_{i}p_{i}c_{i}$

$a=Softmax(W(o+u))$

internal state $u={\sum}_{j}Bq_{j}$

#### 학습: A, C, B, W

#### multi-layer

adjacent: output embedding(o1 + u1)이 새로운 input embedding이 됨   
Layer-wise (RNN-like): 모든 레이어가 A와 C를 공유함.  $uk^{+1} = Hu^{k}+o^{k}$, linear mapping H

#### 성능 올리기: 1. Senetence Representation, 2. Terporal Encoding, 3. Lerning time invariance by injection random noise

In [14]:
print (np.shape(inputs_train))
print (np.shape(queries_train))
print (np.shape(answers_train))

(1000, 10, 6)
(1000, 6)
(1000,)


In [15]:
import mxnet as mx
from mxnet import autograd, gluon, nd, init
from mxnet.gluon import Block, nn


class MemN2N(Block):
    def __init__(self, config, **kwargs):
        super(MemN2N, self).__init__(**kwargs)

        ## 고정
        self.vocab_size = 29
        
        ## 
        self.init_std = 0.015
        self.nhop = 3
        self.emb_dim = 50

        with self.name_scope():
            
            # Embedding A (메모리 벡터)
            self.A = nn.Embedding(input_dim=self.vocab_size, 
                                  output_dim=self.emb_dim, 
                                  weight_initializer=init.Normal(self.init_std))
             
            # Embedding C (컨텍스트 벡터)
            self.C = nn.Embedding(input_dim=self.vocab_size,
                                  output_dim=self.emb_dim, 
                                  weight_initializer=init.Normal(self.init_std))
            
            # Embedding Q (쿼리 벡터)
            self.B = nn.Embedding(input_dim=self.vocab_size, 
                                  output_dim=self.emb_dim, 
                                  weight_initializer=init.Normal(self.init_std))

            # Final Predict 
            self.W = nn.Dense(self.vocab_size, weight_initializer=init.Normal(self.init_std))
            
    def forward(self, sentences, question):

        hid = []
        hid.append(question)
        
        # 16, 10, 7 , 20 > 16, 10, 20
        m_i = self.A(sentences) #* position_encoding(sentences, self.emb_dim)
        m_i = m_i.sum(axis=2)
        
        # 16, 10, 7 , 20 > 16, 10, 20
        c_i = self.C(sentences)
        c_i = c_i.sum(axis=2)
            
        for n in range(self.nhop):
            
            if n == 0:
                #  16, 7, 20 > 16, 20
                u = self.B(hid[-1])# * position_encoding(hid[-1], self.emb_dim)
                u = u.sum(axis=1)
            else:
                u = hid[-1]

            # [16, 1, 20] [16, 20, 10]  > [16, 1, 10]
            m_out = nd.batch_dot(u.expand_dims(1), m_i.swapaxes(1,2))
            P = nd.softmax(m_out, axis=2)
            
            #Cout = P * c_i
            # [16, 1, 10] [16, 10, 20] > [16, 1, 20]
            Cout = nd.batch_dot(P,  c_i)
            # [16, 1, 20] > [16, 20]
            o = Cout.sum(axis=(1))
            
            ## O + U [16, 20]
            Dout = o + u
            hid.append(Dout)
           
        z = self.W(hid[-1])
        return z

In [16]:
def train(model, trainer, x_input, x_query, x_answers, batch_size):
    softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
    max_grad_norm=40
    N = int(math.ceil(len(x_input) / batch_size))
    cost = 0.0

    for idx in range(N):
        with autograd.record():
            out = model(x_input, x_query)
            loss = softmax_cross_entropy(out, x_answers)
            loss.backward()
            
        grads = [i.grad() for i in model.collect_params().values()]
        gluon.utils.clip_global_norm(grads, max_grad_norm)
        trainer.step(batch_size)
        cost += nd.sum(loss).asscalar()

    return cost/N/batch_size

In [17]:
model = MemN2N(None)
model.collect_params().initialize(mx.init.Xavier(),ctx=ctx)

In [39]:
print (model)

MemN2N(
  (C): Embedding(29 -> 50, float32)
  (W): Dense(50 -> 29, linear)
  (A): Embedding(29 -> 50, float32)
  (B): Embedding(29 -> 50, float32)
)


In [19]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.01})
log_loss = []
log_perp = []

In [20]:
batch_size = 32
epochs  = 300

In [21]:
epoch_log_loss = []
import math
for epoch in range(epochs):
    dataiter = mx.io.NDArrayIter([inputs_train, queries_train], answers_train, batch_size, shuffle=False, last_batch_handle='discard')
    
    for batch in dataiter:
        log_loss = []
        train_loss = train(model, trainer, 
                           batch.data[0].as_in_context(ctx), 
                           batch.data[1].as_in_context(ctx),
                           batch.label[0].as_in_context(ctx),
                           batch_size)

        # Logging
        log_loss.append([train_loss])#, test_loss])
    
    epoch_log_loss.append([np.mean(log_loss)])
    
    state = { 'epoch': epoch, 'learning_rate': trainer.learning_rate, 'perplexity': math.exp(epoch_log_loss[epoch][0])}
    print(state)
    
    lr_decay = 1.5
    if (len(epoch_log_loss) > 1) and (epoch_log_loss[epoch][0] > epoch_log_loss[epoch-1][0] * 0.9999):
        print ('update learning rate from %.3f to %.3f' % (trainer.learning_rate, trainer.learning_rate/lr_decay))
        trainer.set_learning_rate(trainer.learning_rate / lr_decay)
    if trainer.learning_rate < 1e-5: 
        break

{'epoch': 0, 'perplexity': 6.03377192163973, 'learning_rate': 0.01}
{'epoch': 1, 'perplexity': 5.575389677322674, 'learning_rate': 0.01}
{'epoch': 2, 'perplexity': 5.550941548354227, 'learning_rate': 0.01}
{'epoch': 3, 'perplexity': 5.175986634233491, 'learning_rate': 0.01}
{'epoch': 4, 'perplexity': 5.54854993218375, 'learning_rate': 0.01}
update learning rate from 0.010 to 0.007
{'epoch': 5, 'perplexity': 3.4855251515202963, 'learning_rate': 0.006666666666666667}
{'epoch': 6, 'perplexity': 2.2255794246030987, 'learning_rate': 0.006666666666666667}
{'epoch': 7, 'perplexity': 1.7860665055961562, 'learning_rate': 0.006666666666666667}
{'epoch': 8, 'perplexity': 1.846349267424668, 'learning_rate': 0.006666666666666667}
update learning rate from 0.007 to 0.004
{'epoch': 9, 'perplexity': 1.6745552927671736, 'learning_rate': 0.0044444444444444444}
{'epoch': 10, 'perplexity': 1.5612030089214397, 'learning_rate': 0.0044444444444444444}
{'epoch': 11, 'perplexity': 1.597875108695894, 'learning_

{'epoch': 92, 'perplexity': 1.2870280934614655, 'learning_rate': 0.0003901844231062339}
{'epoch': 93, 'perplexity': 1.286730136881509, 'learning_rate': 0.0003901844231062339}
{'epoch': 94, 'perplexity': 1.2864358914593927, 'learning_rate': 0.0003901844231062339}
{'epoch': 95, 'perplexity': 1.2861455463311084, 'learning_rate': 0.0003901844231062339}
{'epoch': 96, 'perplexity': 1.2858587539917306, 'learning_rate': 0.0003901844231062339}
{'epoch': 97, 'perplexity': 1.285575397154238, 'learning_rate': 0.0003901844231062339}
{'epoch': 98, 'perplexity': 1.2852955501850487, 'learning_rate': 0.0003901844231062339}
{'epoch': 99, 'perplexity': 1.2850187129639299, 'learning_rate': 0.0003901844231062339}
{'epoch': 100, 'perplexity': 1.2847449984373418, 'learning_rate': 0.0003901844231062339}
{'epoch': 101, 'perplexity': 1.2844738304304255, 'learning_rate': 0.0003901844231062339}
{'epoch': 102, 'perplexity': 1.2842050925383288, 'learning_rate': 0.0003901844231062339}
{'epoch': 103, 'perplexity': 1.

{'epoch': 184, 'perplexity': 1.2664292559948764, 'learning_rate': 0.00011561019943888411}
{'epoch': 185, 'perplexity': 1.266379984076602, 'learning_rate': 0.00011561019943888411}
{'epoch': 186, 'perplexity': 1.26634552695345, 'learning_rate': 0.00011561019943888411}
{'epoch': 187, 'perplexity': 1.2662916919340972, 'learning_rate': 0.00011561019943888411}
{'epoch': 188, 'perplexity': 1.2662465198352344, 'learning_rate': 0.00011561019943888411}
{'epoch': 189, 'perplexity': 1.266188236245751, 'learning_rate': 0.00011561019943888411}
{'epoch': 190, 'perplexity': 1.2661362568680525, 'learning_rate': 0.00011561019943888411}
{'epoch': 191, 'perplexity': 1.2660754503080527, 'learning_rate': 0.00011561019943888411}
{'epoch': 192, 'perplexity': 1.2660194195446484, 'learning_rate': 0.00011561019943888411}
{'epoch': 193, 'perplexity': 1.2659572415060427, 'learning_rate': 0.00011561019943888411}
{'epoch': 194, 'perplexity': 1.2658988769148243, 'learning_rate': 0.00011561019943888411}
{'epoch': 195,

{'epoch': 276, 'perplexity': 1.2608193073300005, 'learning_rate': 0.00011561019943888411}
{'epoch': 277, 'perplexity': 1.2607488743160358, 'learning_rate': 0.00011561019943888411}
{'epoch': 278, 'perplexity': 1.260699917329145, 'learning_rate': 0.00011561019943888411}
{'epoch': 279, 'perplexity': 1.2606291904271951, 'learning_rate': 0.00011561019943888411}
{'epoch': 280, 'perplexity': 1.2605808203953535, 'learning_rate': 0.00011561019943888411}
{'epoch': 281, 'perplexity': 1.2605096118153216, 'learning_rate': 0.00011561019943888411}
{'epoch': 282, 'perplexity': 1.2604617534948617, 'learning_rate': 0.00011561019943888411}
{'epoch': 283, 'perplexity': 1.2603902511402765, 'learning_rate': 0.00011561019943888411}
{'epoch': 284, 'perplexity': 1.260342810524138, 'learning_rate': 0.00011561019943888411}
{'epoch': 285, 'perplexity': 1.2602712585783273, 'learning_rate': 0.00011561019943888411}
{'epoch': 286, 'perplexity': 1.2602241980170468, 'learning_rate': 0.00011561019943888411}
{'epoch': 28

In [31]:
pred_ = model(mx.nd.array(inputs_test, mx.cpu()), mx.nd.array(queries_test, mx.cpu()))
pred_ = pred_.asnumpy()

In [32]:
set([np.argmax(i) for i in pred_])

{2, 3, 5, 6, 10, 13}

In [33]:
print (set(np.argmax(i) for i in pred_))
print (set(i for i in answers_test))

{2, 3, 5, 6, 10, 13}
{2, 3, 5, 6, 10, 13}


In [34]:
print ([np.argmax(i) for i in pred_[0:10]])
print ([i for i in answers_test[0:10]])

[6, 2, 10, 10, 10, 6, 5, 6, 5, 13]
[6, 2, 10, 6, 10, 6, 5, 6, 13, 13]


In [35]:
np.sum([bool(np.argmax(i) == answers_test[idx:idx+1]) for idx, i in enumerate(pred_)])/len(pred_)

0.70699999999999996

In [36]:
## base rate
print (1/len(set(i for i in answers_test)))

0.16666666666666666


In [37]:
idx = 0

In [38]:
for ido in test_data.data_story[idx:idx+1][0]:
    print (" ".join([word_re.get(int(i), '.') for i in ido if int(i) != 0]))
    
print (" ".join([word_re.get(int(i), '.') for i in test_data.data_query[idx:idx+1][0] if int(i) != 0]))
print (word_re.get(int(test_data.data_answer[idx:idx+1][0]), '.'))

john travelled to the hallway mary
mary journeyed to the bathroom kitchen








where is john
hallway


In [30]:
print (word_re[ np.argmax(pred_[idx])])

hallway
